# NEXUS tool: case study for the Jordan - energy demand calculations
In this notebook a case study for the Jordan country is covered using the `nexus_tool` package. The water requirements for agricultural irrigation, residential, industrial and tourism use were previously calculated using the Water Evaluation and Planning System (WEAP) model. In this case study, the energy requirements for groundwater pumping, wastewater treatment, desalination of seawater and pumping for water conveyance are estimated.

First import the package by running the following block:

In [ ]:
import sys
sys.path.append("..") #this is to add the avobe folder to the package directory
import os
import nexus_tool
import pandas as pd

## 1. Read scenario data
After importing all required packages, the input GIS data is loaded into the variable `df`. Change the `data_folder`, `scenario`, `climate` and `level` variables to reflect the name and relative location of your data file. This dataset should already have the water demand for irrigation data.

In [ ]:
data_folder = os.path.join('Data', 'Processed results')
scenario = 'Reference'
climate = 'Climate Change'
level = 'level_1'
input_folder = os.path.join(data_folder, scenario, climate, level)

## 2. Create nexus model 
To create a model simply create an instance of the `nexus_tool.Model()` class and store it in a variable name. The `nexus_tool.Model()` class requires a dataframe as input data. Several other properties and parameter values can be defined by explicitly passing values to them. To see a full list of parameters and their explaination refer to the documentation of the package. We wil create a model using the `pipelines_flow.csv` data:

In [ ]:
file_path = os.path.join(input_folder, 'pipelines_flow.csv')
df = nexus_tool.read_csv(file_path)
jordan = nexus_tool.Model(df)

## 3. Define variable names
The names of the properties of the model can be changed at any time. This is important for the model to know how each property is called withing your input data. To check the current property names run the `.print_properties()` method, a list with the names of each property and its current value will be displayed.

Then you can provide the right names for each property, calling them and assigning a value as:
```python
jordan.elevation = 'elevation_delta'
jordan.gw_depth = 'name_of_ground_water_depth'
```

In this particular case we will need to change the following default values:

In [ ]:
jordan.elevation = 'elevation_delta'
jordan.L = 'segment_length'
jordan.peak_Q = 'value'
jordan.avg_Q = 'value'

## 4. Define pipelines diameters and pumping efficiency
Now we need to define the specifications of the water network, giving pipeline / canal diameter values and surface pumping efficiency:

In [ ]:
jordan.df['Pipe_diameter'] = 1 #in m
jordan.df.loc[(df['pipeline']=='KAC'), 'Pipe_diameter'] = 4 #in m
jordan.df.loc[(df['pipeline']=='PL_Disi'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PL_RedDead'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PS_ZaraMain'), 'Pipe_diameter'] = 1.8 #in m
jordan.df.loc[(df['pipeline']=='PL_KACToZay'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PL_Dab_AinGhazal'), 'Pipe_diameter'] = 1.4 #in m
jordan.df.loc[(df['pipeline']=='PL_ZaytoDabouq'), 'Pipe_diameter'] = 1.4 #in m

jordan.SWpump_eff = 0.5 # pumping efficiency

## 5. Calculate pumping energy requirements
To estimate the pumping energy requirements for conveyance, first we need to calculate the Total Dinamic Head (TDH). This, is a measure in meters that accounts for the elevation difference between two points and the pressure loss in distribution.

For that, the area `A`, the velocity `V`, the Reynolds number `Re` and the friction factor `f` need to be estimated. The `nexus_tool` provides simple functions that will allow for an easy estimation of these variables. Then, the TDH can be calculated by simply calling the `get_sw_tdh()` function, whereas the conveyance pumping energy requirements by calling the `nwsas.get_GWpumping_energy()` method.

In [ ]:
jordan.get_A(inplace=True)
jordan.get_V(inplace=True, axis=0)
jordan.get_Re(inplace=True, axis=0)
jordan.get_f(inplace=True, axis=0)

jordan.get_sw_tdh(inplace = True, axis=0)
jordan.get_SWpumping_energy(inplace = True, axis=0)

jordan.df.loc[(df['pipeline']=='KAC'), 'SWPA_E_'] = 0 #Setting energy for canal to 0

## 6. Create nexus model for groundwater pumping
The steps needed to estimate the groundwater pumping energy requirements are farily similar to the previous ones. We need to set the path to the `groundwater_supply.csv`, read it into a dataframe and create a nexus Model with it. Then, we define the pipe diameter and some required atttributes names:

In [ ]:
file_path = os.path.join(input_folder, 'groundwater_supply.csv')
df_groundwater = pd.read_csv(file_path)

jordan_gw = nexus_tool.Model(df_groundwater)

jordan_gw.df['Pipe_diameter'] = 0.4
jordan_gw.elevation = 'wtd_m'
jordan_gw.L = 'wtd_m'
jordan_gw.peak_Q = 'value'
jordan_gw.avg_Q = 'value'

## 7. Calcuculate groundwater pumping energy
The energy requirements are then calculated calling the `get_A`, `get_V`, `get_Re`, `get_f`, `get_sw_tdh` and get `SWpumping_energy` methods.

In [ ]:
jordan_gw.get_A(inplace=True)
jordan_gw.get_V(inplace=True, axis=0)
jordan_gw.get_Re(inplace=True, axis=0)
jordan_gw.get_f(inplace=True, axis=0)

jordan_gw.get_sw_tdh(inplace = True, axis=0)
jordan_gw.get_SWpumping_energy(inplace = True, axis=0)

## 8. Calculate wastewater treatment energy
To estimate the energy requirements for wastewater treatment, we read the `wwtp_inflow.csv` data into a dataframe and multiply the `value` attribute, wich represents the amount of treated water, by a defined energy intensity representative of the treatment process.

In [ ]:
file_path = os.path.join(input_folder, 'wwtp_inflow.csv')
df_wwtp = pd.read_csv(file_path)

wwtp_energy_int = 0.6 # kWh/m3
df_wwtp['SWPA_E_'] = df_wwtp.value * wwtp_energy_int

## 9. Calculate desalination energy
Finally, we need to calculate the energy requirements for desalination. Thus we read the `desalination.csv` data into a dataframe and similarly to the wastewater treatment case, we multiply the desalinated water `value` by each desalination plant, by a respective energy intensity value.

In [ ]:
file_path = os.path.join(input_folder, 'desalination.csv')
df_desal = pd.read_csv(file_path)

red_dead_energy_int = 3.31 # kWh/m3
aqaba_energy_int = 5 # kWh/m3
df_desal['SWPA_E_'] = 0

df_desal.loc[df_desal.point=='RedDead', 'SWPA_E_'] = df_desal.loc[df_desal.point=='RedDead', 'value'] * red_dead_energy_int
df_desal.loc[df_desal.point=='Aqaba Desal', 'SWPA_E_'] = df_desal.loc[df_desal.point=='Aqaba Desal', 'value'] * aqaba_energy_int

## 10. Save result files
The results can then be saved into a defined output folder `results_folder`:

In [ ]:
results_folder = os.path.join('..','Jordan dashboard', 'data_test', scenario, climate, level)
os.makedirs(results_folder, exist_ok=True)

jordan.df.to_csv(os.path.join(results_folder, 'pipelines_data.csv'), index=False)
jordan_gw.df.to_csv(os.path.join(results_folder, 'groundwater_pumping.csv'), index=False)
df_wwtp.to_csv(os.path.join(results_folder, 'wwtp_data.csv'), index=False)
df_desal.to_csv(os.path.join(results_folder, 'desal_data.csv'), index=False)